In [28]:
import pandas as pd
import pickle 
import xml.etree.cElementTree as ET
from bs4 import BeautifulSoup
import requests
import urllib
import os
import zipfile
import lxml
from lxml import etree

In [24]:
os.chdir('/work/')

# 1. Data Download


In [22]:
os.chdir('notebook')

In [23]:
%load_ext autoreload
%autoreload 2
import patent_utils as pu

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
year = 2005
lnk = 'https://bulkdata.uspto.gov/data/patent/grant/redbook/fulltext/'\
          + str(year) + '/'
target_dir = '/work/data/grants/' + str(year) + '/'
pu.download_all_links(lnk, target_dir)

In [18]:
os.getcwd()


'/work'

In [ ]:
for year in list(range(2005, 2012, 1)):
    lnk = 'https://bulkdata.uspto.gov/data/patent/grant/redbook/fulltext/'\
          + str(year) + '/'
    target_dir = '/work/data/grants/' + str(year) + '/'
    pu.download_all_links(lnk, target_dir)

In [ ]:
for year in  list(range(2005, 2012, 1)):
    newdir = '/work/data/grants/' + str(year) + '/'
    os.chdir(newdir)
    split_grants()
    get_details_from_all_folders(newdir, year)

ipg061024.xml
ipg060627.xml
ipg060110.xml
ipg060718.xml
ipg060905.xml
ipg060516.xml
ipg060509.xml
ipg061128.xml
ipg060221.xml
ipg060815.xml
ipg060117.xml
ipg061219.xml
ipg060207.xml
ipg060530.xml
ipg061205.xml
ipg060829.xml
ipg060328.xml
ipg060704.xml
ipg061031.xml
ipg061017.xml
ipg060606.xml
ipg060502.xml
ipg060214.xml
ipg060321.xml
ipg060919.xml
ipg060103.xml
ipg060912.xml
ipg060124.xml
ipg060314.xml
ipg060425.xml
ipg060725.xml
ipg060822.xml
ipg060711.xml
ipg061212.xml
ipg060307.xml
ipg061003.xml
ipg060808.xml
ipg060404.xml
ipg060228.xml
ipg060523.xml
ipg060411.xml
ipg061010.xml
ipg061107.xml
ipg060131.xml
ipg061226.xml
ipg060926.xml
ipg060418.xml
ipg061121.xml
ipg060801.xml
ipg060613.xml
ipg061114.xml
ipg060620.xml
ipg070123.xml
ipg070703.xml
ipg071002.xml
ipg070109.xml
ipg070710.xml
ipg070522.xml
ipg070306.xml
ipg071120.xml
ipg070619.xml
ipg070102.xml
ipg071016.xml
ipg070417.xml
ipg071023.xml
ipg071225.xml
ipg071211.xml
ipg070220.xml
ipg070130.xml
ipg070501.xml
ipg070612.xml
ipg071

In [4]:
year = 2012
#download_applications(year)
olddir = os.getcwd()
newdir = '/work/data/grants/' + str(year) + '/'
os.chdir(newdir)

# 2. Split Files

In [30]:
def unzip_file(fnm):
    zip_ref = zipfile.ZipFile(fnm, 'r')
    zip_ref.extractall('.')
    zip_ref.close()
    return fnm.split('/')[-1].replace('.zip', '.xml')

def split_apps():
    for fnm in [x for x in os.listdir() if x.split('.')[-1]=='zip']:
        fnm_xml = unzip_file(fnm)
        print(fnm_xml)
        !bash /work/split_apps.sh {fnm_xml}
        !rm {fnm}
        
def split_grants():
    for fnm in [x for x in os.listdir() if x.split('.')[-1]=='zip']:
        fnm_xml = unzip_file(fnm)
        print(fnm_xml)
        !bash /work/split_grants.sh {fnm_xml}
        !rm {fnm}

# 3. Meta Data Extraction

In [31]:
def extract_singles_from_xpath(tree, xpath):
    fsub = tree.xpath(xpath)
    if fsub == []:
        return ''
    return ''.join(fsub[0].itertext())

def extract_dict_from_xpath(tree, xpath):
    fsub = tree.xpath(xpath)
    if fsub == []:
        return {}
    ref_dict = {x.tag: x.text for x in fsub[0].iterchildren()}
    ref_dict.update({'len':len(fsub)})
    return ref_dict

def extract_all_singles_from_xpath(tree, xpaths):
    out = {v:extract_singles_from_xpath(tree, k) for k, v in xpaths.items()}
    return out
    
def extract_all_xpath_dicts(tree, xpaths):
    keys = []
    values = []
    
    for k, v in xpaths.items():
        edict = extract_dict_from_xpath(tree, k)
        keys.extend([v + '_' + x for x in edict.keys()])
        values.extend(list(edict.values()))
    
    edict = dict(zip(keys, values))
    return edict

In [32]:
def get_details_from_xml(fnm):
    tree = etree.parse(fnm)
    xpaths_singles = {
        '/us-patent-grant/us-bibliographic-data-grant/'+\
        'invention-title': 'title',
    }
    
    xpaths_dicts = {
        '/us-patent-grant/us-bibliographic-data-grant/'+\
        'application-reference/document-id': 'app',
        '/us-patent-grant/us-bibliographic-data-grant/'+\
        'publication-reference/document-id': 'pub'
    }
    out_dict = extract_all_singles_from_xpath(tree, xpaths_singles)
    out_dict.update(extract_all_xpath_dicts(tree, xpaths_dicts))
    out_dict.update({'fnm':fnm})
    return out_dict

def get_details_from_folder(file_dir):
    dict_list = [get_details_from_xml(file_dir + x) for x in 
                                     os.listdir(file_dir)]
    df = pd.DataFrame.from_dict(dict_list)
    return df

In [33]:
def get_details_from_all_folders(basedir, year):
    df = [get_details_from_folder(basedir + x + '/') for x in 
           os.listdir(basedir)]
    df = pd.concat(df)
    df.to_csv('/work/data/grants/' + str(year) + '_all_grants.csv')
    return df
    

In [18]:
get_details_from_all_folders('/work/data/grants/2012/')

NameError: name 'etree' is not defined

In [ ]:
def make_year(year):
    download_applications(year)
    olddir = os.getcwd()
    newdir = '/work/data/apps/' + str(year) + '/'
    os.chdir(newdir)
    split_files()
    get_details_from_folders(newdir)
    return meta_df

In [9]:
%%time
year = 2012

newdir = '/work/data/grants/' + str(year) + '/'
os.chdir(newdir)
split_grants()

ipg120717.xml
ipg121225.xml
ipg120703.xml
ipg120417.xml
ipg120731.xml
ipg121009.xml
ipg120918.xml
ipg120501.xml
ipg121030.xml
ipg120207.xml
ipg120124.xml
ipg120221.xml
ipg120403.xml
ipg120814.xml
ipg121218.xml
ipg121204.xml
ipg121211.xml
ipg120327.xml
ipg120710.xml
ipg120508.xml
ipg120214.xml
ipg121023.xml
ipg120103.xml
ipg121113.xml
ipg121120.xml
ipg120515.xml
ipg120228.xml
ipg120612.xml
ipg120911.xml
ipg120410.xml
ipg121106.xml
ipg120904.xml
ipg120522.xml
ipg120110.xml
ipg121002.xml
ipg120626.xml
ipg120925.xml
ipg120313.xml
ipg120828.xml
ipg120131.xml
ipg120821.xml
ipg120807.xml
ipg120529.xml
ipg120724.xml
ipg120605.xml
ipg121016.xml
ipg120306.xml
ipg120117.xml
ipg120619.xml
ipg120424.xml
ipg120320.xml
ipg121127.xml
CPU times: user 2min 10s, sys: 17.2 s, total: 2min 27s
Wall time: 5min 37s


# 2. Find doc-numbers

In [4]:
def get_docnumbers(fnm):
    #try:
        tree = ET.parse(fnm)
        dc_nums = ["".join([x for x in numb.itertext()]) for numb in 
                                           tree.iter(tag = 'doc-number')]
        return pd.DataFrame.from_dict({fnm.split('/')[-1] : dc_nums}).T
    #except:
    #    return pd.DataFrame.from_dict({fnm.split('/')[-1] : [-1111]}).
    
    
def find_app_ids(file_dir):
    #from IPython.core.debugger import Tracer; Tracer()()
    rows = [get_docnumbers(file_dir + x) for x in 
                                     os.listdir(file_dir)]
    pdrows = pd.concat(rows)
    pdrows.index = file_dir + pdrows.index
    return pdrows

In [17]:
%%time
all_app_ids = pd.concat([find_app_ids(newdir + x +'/') 
                              for x in os.listdir(newdir)])
all_app_ids = all_app_ids.rename(columns={1: 'app_id'})
all_app_ids.to_csv(str(year) + '_app_fnm_and_ids.csv')

CPU times: user 36min 36s, sys: 59.2 s, total: 37min 35s
Wall time: 1h 3min 13s


In [20]:
all_app_ids.info()

<class 'pandas.core.frame.DataFrame'>
Index: 374732 entries, /work/data/apps/2017/ipa171221/F_1722.xml to /work/data/apps/2017/ipa170406/F_2924.xml
Columns: 776 entries, 0 to 775
dtypes: object(776)
memory usage: 2.2+ GB


# 3. Looking at the doc-numbers

In [9]:
%%time
all_app_ids = pd.read_csv('2017_app_fnm_and_ids.csv', low_memory=False)

CPU times: user 42.9 s, sys: 5.06 s, total: 48 s
Wall time: 48 s


In [ ]:
def get_docnumbers(fnm):
    #try:
        tree = ET.parse(fnm)
        dc_nums = ["".join([x for x in numb.itertext()]) for numb in 
                                           tree.iter(tag = 'doc-number')]
        return pd.DataFrame.from_dict({fnm.split('/')[-1] : dc_nums}).T

In [5]:
fnm = '/work/data/apps/2017/ipa171221/F_1722.xml'
tree = ET.parse(fnm)


In [8]:
from lxml import etree
tree = etree.parse(fnm)

In [11]:
grants = pd.read_csv('../2012_all_grants.csv')

In [13]:
grants.shape

(277285, 12)

In [13]:
def extract_singles_from_xpath(tree, xpath):
    fsub = tree.xpath(xpath)
    if fsub == []:
        return ''
    return ''.join(fsub[0].itertext())

def extract_dict_from_xpath(tree, xpath):
    fsub = tree.xpath(xpath)
    if fsub == []:
        return {}
    ref_dict = {x.tag: x.text for x in fsub[0].iterchildren()}
    ref_dict.update({'len':len(fsub)})
    return ref_dict

def extract_all_singles_from_xpath(tree, xpaths):
    out = {v:extract_singles_from_xpath(tree, k) for k, v in xpaths.items()}
    return out
    
def extract_all_xpath_dicts(tree, xpaths):
    keys = []
    values = []
    
    for k, v in xpaths.items():
        edict = extract_dict_from_xpath(tree, k)
        keys.extend([v + '_' + x for x in edict.keys()])
        values.extend(list(edict.values()))
    
    edict = dict(zip(keys, values))
    return edict

In [200]:
def get_details_from_xml(fnm):
    tree = etree.parse(fnm)
    xpaths_singles = {
        '/us-patent-application/us-bibliographic-data-application/assignees/'+\
        'assignee/addressbook/orgname': 'assignee',
        '/us-patent-application/us-bibliographic-data-application/'+\
        'invention-title': 'title',
        '/us-patent-application/us-bibliographic-data-application/'+\
        'us-parties/us-applicants/us-applicant/addressbook/orgname': 
        'us-applicant'
    }
    
    xpaths_dicts = {
        '/us-patent-application/us-bibliographic-data-application/'+\
        'application-reference/document-id': 'app',
        '/us-patent-application/us-bibliographic-data-application/'+\
        'publication-reference/document-id': 'pub'
    }
    out_dict = extract_all_singles_from_xpath(tree, xpaths_singles)
    out_dict.update(extract_all_xpath_dicts(tree, xpaths_dicts))
    out_dict.update({'fnm':fnm})
    return out_dict
    

In [183]:
from tqdm import tqdm

In [217]:
def get_details_from_folder(file_dir):
    #from IPython.core.debugger import Tracer; Tracer()()
    dict_list = [get_details_from_xml(file_dir + x) for x in 
                                     os.listdir(file_dir)]
    df = pd.DataFrame.from_dict(dict_list)
    return df



  7%|▋         | 529/7150 [00:29<06:05, 18.14it/s]

In [202]:
%%time
all_df = get_details_from_folder('/work/data/apps/2017/ipa170202/')

100%|██████████| 7885/7885 [00:14<00:00, 541.15it/s]

CPU times: user 14 s, sys: 694 ms, total: 14.7 s
Wall time: 14.6 s


In [ ]:
basedir = '/work/data/apps/2017/'
dfs = [get_details_from_folder(basedir + x + '/') for x in os.listdir(basedir)]
pd.concat(dfs).to_csv('/work/data/apps/2017_all_apps.csv')

In [10]:
all_apps = pd.read_csv('/work/data/apps/2017_all_apps.csv', low_memory=False).iloc[:, 1:]
all_apps.head()

,app_country,app_date,app_doc-number,app_len,assignee,fnm,pub_country,pub_date,pub_doc-number,pub_kind,pub_len,title,us-applicant
0,US,20151203,15540631,1,"Bridgestone Americas Tire Operations, LLC",/work/data/apps/2017/ipa171221/F_1722.xml,US,20171221,20170361662,A1,1,RADAR WEAR SENSING FOR TIRE APPLICATIONS,"Bridgestone Americas Tire Operations, LLC"
1,US,20151204,15532203,1,NaN,/work/data/apps/2017/ipa171221/F_959.xml,US,20171221,20170360899,A1,1,TREATING SEIZURE WITH RECOMBINANT ALKALINE PHO...,"Alexion Pharmaceuticals, Inc."
2,US,20151203,15532255,1,NaN,/work/data/apps/2017/ipa171221/F_1012.xml,US,20171221,20170360952,A1,1,BIOMOLECULE CONJUGATES,CELGENE CORPORATION
3,US,20151203,15533313,1,NaN,/work/data/apps/2017/ipa171221/F_2534.xml,US,20171221,20170362474,A1,1,Bonding Method and Conveyor Belt,"The Yokohama Rubber Co., Ltd."
4,US,20151207,15533160,1,NaN,/work/data/apps/2017/ipa171221/F_1033.xml,US,20171221,20170360973,A1,1,ELECTROCHEMICAL DEVICE FOR RELEASING IONS,NMR Technology AS


In [12]:
all_apps.loc[all_apps['app_doc-number'].isnull()]

,app_country,app_date,app_doc-number,app_len,assignee,fnm,pub_country,pub_date,pub_doc-number,pub_kind,pub_len,title,us-applicant


In [32]:
oas = pd.read_csv('/work/data/office_actions.csv', low_memory=False,
                  parse_dates=['mail_dt'])

In [47]:
cits = pd.read_csv('/work/data/citations.csv', low_memory=False)

In [48]:
cits.head().T

,0,1,2,3,4
app_id,12000001,12000001,12000001,12000001,12000001
citation_pat_pgpub_id,20060218340,2343564,6622200,6182004,2323432
parsed,20060218340,2343564,6622200,6182004,2323432
ifw_number,NaN,NaN,NaN,NaN,NaN
action_type,NaN,NaN,NaN,NaN,NaN
action_subtype,NaN,NaN,NaN,NaN,NaN
form892,0,1,1,0,1
form1449,1,0,0,1,0
citation_in_oa,0,0,0,0,0


In [15]:
oas.index.shape[0] - oas.index.unique().shape[0]

85598

In [18]:
oas.app_id.shape[0] - oas.app_id.unique().shape[0]

2196493

In [14]:
oas.head()

,,ifw_number,document_cd,art_unit,uspc_class,uspc_subclass,header_missing,fp_missing,rejection_fp_mismatch,closing_missing,rejection_101,...,rejection_103,rejection_112,rejection_dp,objection,allowed_claims,cite102_gt1,cite103_gt3,cite103_eq1,cite103_max,signature_type
app_id,mail_dt,,,,,,,,,,,,,,,,,,,,,
14150981,2015-10-15,100867762,CTFR,2632,375,219000,0,0,0,0,0,...,1,0,0,0,0,0,0,1,2,1
14198961,2015-10-15,100867788,CTFR,2699,345,173000,0,0,0,0,0,...,1,0,0,0,0,0,0,0,2,1
13796589,2015-10-15,100867794,CTNF,3776,606,159000,0,0,0,0,0,...,1,0,0,0,0,0,0,0,3,3
14673475,2015-10-15,100867844,CTNF,2627,345,175000,0,0,0,0,0,...,1,1,1,0,0,0,1,0,4,1
14669113,2015-10-15,100867850,CTNF,2666,382,118000,0,0,0,0,1,...,0,1,1,0,0,0,0,0,0,1


In [24]:
oas = oas.sort_values('app_id')

In [ ]:
oas = oas.set_index(['app_id', 'mail_dt'])

In [ ]:
oas.set

In [17]:
%matplotlib inline

In [25]:
(oas.app_id.value_counts().value_counts().transform(lambda x: x/x.sum())*100).to_frame('app_id_count (%)')

,app_id_count (%)
1,48.408324
2,27.533650
3,10.897795
4,7.081272
5,2.994873
6,1.738863
7,0.726450
8,0.362196
9,0.142822
10,0.068966


In [23]:
all_app_ids.app_id.unique().shape

(374472,)

In [33]:
dupes = all_app_ids.loc[all_app_ids.app_id.duplicated(keep=False)].\
                    sort_values('app_id')

In [41]:
oas_2017_dupes = oas.loc[(oas.app_id.isin(dupes.app_id)) & (oas.mail_dt.dt.year==2017)]

In [43]:
oas_2017_dupes.sort_values('app_id')

,app_id,ifw_number,document_cd,mail_dt,art_unit,uspc_class,uspc_subclass,header_missing,fp_missing,rejection_fp_mismatch,...,rejection_103,rejection_112,rejection_dp,objection,allowed_claims,cite102_gt1,cite103_gt3,cite103_eq1,cite103_max,signature_type
4228910,14336505,IXTQIFZVRXEAPX2,CTFR,2017-01-13,2166,707,005000,0,0,0,...,1,0,0,0,0,0,0,0,3,1
4333343,14336505,J3U4QJ0CRXEAPX2,CTNF,2017-06-14,2166,707,005000,0,0,0,...,1,0,0,0,0,0,0,0,3,2
4353444,14409128,J46YFHLURXEAPX2,CTNF,2017-06-23,1767,252,182320,0,0,0,...,1,0,0,0,0,0,0,1,1,2
4352222,14640075,J45P9SL2RXEAPX3,CTFR,2017-06-20,1645,424,234100,0,0,0,...,1,1,0,1,0,0,0,1,2,2
4338049,14727523,J3VU579KRXEAPX1,CTNF,2017-06-15,2687,340,010400,0,0,0,...,0,0,1,1,1,0,0,0,0,0
4331642,14756008,J3PVHAA3RXEAPX0,CTNF,2017-06-09,3765,002,149000,0,0,0,...,1,1,0,1,0,0,0,0,2,1
4309476,14795872,IZJXURHGRXEAPX3,CTNF,2017-02-24,1645,424,190100,0,1,0,...,1,0,0,0,0,1,1,0,4,0
4281200,14800594,IYX3JLJBRXEAPX0,CTNF,2017-02-10,2178,715,243000,0,0,0,...,1,0,0,0,0,0,1,0,6,3
4255896,14800628,IYDFUDDZRXEAPX2,CTNF,2017-01-27,2178,715,231000,0,0,0,...,1,0,0,0,0,0,0,1,2,3
4300983,14803590,IZFW7E8MRXEAPX1,CTFR,2017-02-23,0,455,041100,0,0,0,...,1,0,0,1,0,0,0,0,3,2


In [45]:
oas.loc[oas.app_id.isin(['14336505'])]

,app_id,ifw_number,document_cd,mail_dt,art_unit,uspc_class,uspc_subclass,header_missing,fp_missing,rejection_fp_mismatch,...,rejection_103,rejection_112,rejection_dp,objection,allowed_claims,cite102_gt1,cite103_gt3,cite103_eq1,cite103_max,signature_type
2510706,14336505,I0I84V4UPXXIFW4,CTNF,2014-09-29,2166,707,005000,0,0,0,...,1,1,1,0,0,0,0,0,2,0
2781712,14336505,I6104XE0PXXIFW4,CTFR,2015-02-13,2166,707,005000,0,0,0,...,1,1,0,0,0,0,0,0,3,1
3266925,14336505,IEWY8NO7PXXIFW4,CTFR,2015-09-25,2166,707,005000,0,0,0,...,1,0,0,0,0,0,0,0,3,1
3972465,14336505,ISKFPTR8RXEAPX1,CTNF,2016-09-06,2166,707,005000,0,0,0,...,1,1,0,0,0,0,0,0,3,1
4228910,14336505,IXTQIFZVRXEAPX2,CTFR,2017-01-13,2166,707,005000,0,0,0,...,1,0,0,0,0,0,0,0,3,1
4333343,14336505,J3U4QJ0CRXEAPX2,CTNF,2017-06-14,2166,707,005000,0,0,0,...,1,0,0,0,0,0,0,0,3,2


In [ ]:
cits = pd.read

In [44]:
dupes.query('app_id == "14336505"')

,Unnamed: 0,0,app_id,2,3,4,5,6,7,8,...,766,767,768,769,770,771,772,773,774,775
304678,/work/data/apps/2017/ipa171019/F_4814.xml,20170300526,14336505,20140330864,20160267130,20170147634,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
186363,/work/data/apps/2017/ipa170525/F_4744.xml,20170147634,14336505,20140330864,20160267130,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,Unnamed: 0,0,app_id,2,3,4,5,6,7,8,...,766,767,768,769,770,771,772,773,774,775
78073,/work/data/apps/2017/ipa170202/F_2073.xml,20170029169,13671978,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
87100,/work/data/apps/2017/ipa170720/F_1749.xml,20170203877,13671978,61557168,20170029169,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
212829,/work/data/apps/2017/ipa170427/F_752.xml,20170112793,14209480,11576871,PCT/DK05/00648,14209480,60691513,20140193495,20160136125,20160271093,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
109197,/work/data/apps/2017/ipa170119/F_599.xml,20170014367,14209480,11576871,PCT/DK05/00648,14209480,60691513,20140193495,20160136125,20160271093,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73571,/work/data/apps/2017/ipa170706/F_7177.xml,20170195681,14311083,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
oas.mail_dt.dt.year

0          2015
1          2015
2          2015
3          2015
4          2015
5          2015
6          2015
7          2015
8          2015
9          2015
10         2015
11         2015
12         2015
13         2015
14         2015
15         2015
16         2015
17         2015
18         2015
19         2015
20         2015
21         2015
22         2015
23         2015
24         2015
25         2015
26         2015
27         2015
28         2015
29         2015
           ... 
4384502    2017
4384503    2017
4384504    2017
4384505    2017
4384506    2017
4384507    2017
4384508    2017
4384509    2017
4384510    2017
4384511    2017
4384512    2017
4384513    2017
4384514    2017
4384515    2017
4384516    2017
4384517    2017
4384518    2017
4384519    2017
4384520    2017
4384521    2017
4384522    2017
4384523    2017
4384524    2017
4384525    2017
4384526    2017
4384527    2017
4384528    2017
4384529    2017
4384530    2017
4384531    2017
Name: mail_dt, Length: 4